# Projet : Systeme de recommandation d'images

## Presentation

Dans ce projet, vous allez construire un **systeme de recommandation d'images** qui suggere des images aux utilisateurs en fonction de leurs preferences. Ce projet met en pratique toutes les competences acquises lors des seances pratiques : analyse de donnees, visualisation, regroupement, classification et apprentissage automatique.

**Duree** : 3 seances pratiques
**Taille de l'equipe** : 2-3 etudiants
**Livrables** :
1. Un notebook Jupyter (`Nom1_Nom2_[Nom3].ipynb`)
2. Un rapport de synthese de 4 pages (PDF)

---

## Objectifs d'apprentissage

En completant ce projet, vous serez capable de :
- Automatiser la collecte de donnees a partir de sources web
- Extraire et traiter les metadonnees d'images
- Appliquer des algorithmes de regroupement pour analyser les caracteristiques des images
- Construire des profils de preferences utilisateur
- Implementer un algorithme de recommandation
- Visualiser efficacement les donnees
- Ecrire des tests complets pour votre systeme

---

## Architecture du projet

Le systeme est compose de 7 taches interconnectees :

```
┌─────────────────────────────────────────────────────────────────┐
│               SYSTEME DE RECOMMANDATION D'IMAGES                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐       │
│  │ 1. Collecte  │───▶│ 2. Etiquetage│───▶│ 3. Analyse   │       │
│  │ de donnees   │    │ & Annotation │    │ de donnees   │       │
│  └──────────────┘    └──────────────┘    └──────────────┘       │
│        │                    │                   │                │
│        ▼                    ▼                   ▼                │
│  ┌──────────────────────────────────────────────────────┐       │
│  │          Fichiers JSON (Stockage des metadonnees)     │       │
│  └──────────────────────────────────────────────────────┘       │
│        │                    │                   │                │
│        ▼                    ▼                   ▼                │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐       │
│  │ 4. Visuali-  │    │ 5. Systeme de│    │ 6. Tests     │       │
│  │ sation       │    │ recommandat. │    │              │       │
│  └──────────────┘    └──────────────┘    └──────────────┘       │
│                             │                                    │
│                             ▼                                    │
│                    ┌──────────────┐                              │
│                    │ 7. Rapport   │                              │
│                    │ de synthese  │                              │
│                    └──────────────┘                              │
└─────────────────────────────────────────────────────────────────┘
```

![Architecture](../../images/Project-Architecture.png "Architecture")

---

## Projet partie 1

---

### Tache 1 : Collecte de donnees

#### Objectif
Collecter au moins **100 images sous licence ouverte** avec leurs metadonnees.

#### Ce que vous devez faire

1. **Creer une structure de dossiers** :
   ```
   projet/
   ├── images/           # Images telechargees
   ├── data/             # Fichiers de metadonnees JSON
   └── projet.ipynb      # Votre notebook
   ```

2. **Trouver des sources d'images** (une ou plusieurs) :
   - [Wikimedia Commons](https://commons.wikimedia.org/) - Utilisez des requetes SPARQL (comme dans le TP 1)
   - [Unsplash API](https://unsplash.com/developers) - API gratuite pour des images de haute qualite
   - [Pexels API](https://www.pexels.com/api/) - Photos libres de droits
   - [Flickr API](https://www.flickr.com/services/api/) - Images Creative Commons

3. **Telecharger les images par programme** en utilisant les techniques du TP 1, Exercice 6

4. **Extraire et sauvegarder les metadonnees** de chaque image :
   - Nom du fichier image
   - Dimensions de l'image (largeur, hauteur)
   - Format du fichier (.jpg, .png, etc.)
   - Taille du fichier (en Ko)
   - URL source
   - Informations de licence
   - Donnees EXIF (si disponibles) : modele d'appareil photo, date de prise de vue, etc.

#### Resultat attendu
- Dossier `images/` contenant 100+ images
- `data/images_metadata.json` contenant les metadonnees de toutes les images

#### Conseils
- Utilisez `PIL` pour obtenir les dimensions de l'image
- Utilisez `os.path.getsize()` pour obtenir la taille du fichier
- Utilisez l'extraction EXIF (voir TP 2, Exercice 2)
- Stockez les metadonnees sous forme de liste de dictionnaires au format JSON

---

## Tâche 1 — Collecte de données : téléchargement des images
On récupère la liste des photos depuis l'API `jmail.world`, puis on télécharge les 100 premières dans `images/`.

In [4]:
import requests, time
import json
from pathlib import Path

# ── Config ─────────────────────────────────────────────────────
LIMIT = 100
API_URL = "https://jmail.world/api/photos?newOnly=false"
CDN_BASE = "https://assets.getkino.com/photos-deboned"
IMAGES_DIR = Path("images")
DATA_DIR = Path("data")
IMAGES_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)

# ── 1. Récupérer la liste des photos depuis l'API ─────────────
session = requests.Session()
resp = session.get(API_URL, timeout=30)
resp.raise_for_status()
all_photos = resp.json()["photos"]

# Filtrer : garder seulement les images AVEC source_url (celles-ci sont dispos)
photos = [p for p in all_photos if p.get("source_url")][:LIMIT]
print(f"API : {len(all_photos)} photos total, {len(photos)} disponibles sur CDN")

# ── 2. Télécharger chaque image dans images/ ──────────────────
downloaded = 0
for i, photo in enumerate(photos, 1):
    filename = photo["original_filename"]
    dest = IMAGES_DIR / filename
    if dest.exists():
        downloaded += 1
        continue
    url = f"{CDN_BASE}/{filename}"
    try:
        r = session.get(url, stream=True, timeout=60)
        r.raise_for_status()
        with open(dest, "wb") as f:
            for chunk in r.iter_content(8192):
                f.write(chunk)
        downloaded += 1
    except Exception as e:
        print(f"  ✗ {filename}: {e}")
    if i % 20 == 0:
        print(f"  [{i}/{len(photos)}]…")
    time.sleep(0.05)

# ── 3. Sauvegarder les métadonnées JSON ────────────────────────
metadata_file = DATA_DIR / "image_metadata.json"
with open(metadata_file, "w", encoding="utf-8") as f:
    json.dump(photos, f, ensure_ascii=False, indent=2)

print(f"\n✓ {downloaded}/{len(photos)} images dans {IMAGES_DIR}/")
print(f"✓ Métadonnées sauvegardées dans {metadata_file}")

API : 18316 photos total, 100 disponibles sur CDN
  ✗ EFTA00004157-3.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00004157-3.png
  ✗ EFTA00004157-5.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00004157-5.png
  ✗ EFTA00004157-7.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00004157-7.png
  ✗ EFTA00005381-1.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00005381-1.png
  ✗ EFTA00005386-78.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00005386-78.png
  ✗ EFTA00005569-1.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00005569-1.png

✓ 94/100 images dans images/
✓ Métadonnées sauvegardées dans data\image_metadata.json


In [7]:
!pip install Pillow exifread

In [21]:
from PIL import Image
import os
import pandas as pd

paletteListe = []
modeListe = []
tailleListe = []


def display_image_metadata(image_path):
    """Affiche les métadonnées générales d'une image."""
    with Image.open(image_path) as img:
        print("\nMétadonnées de l'image :")
        print(f"Format : {img.format}")
        print(f"Mode : {img.mode}")
        print(f"dimension : {img.size}")
        print(f"Palette : {img.palette}")
        print(f"Infos supplémentaires : {img.info}")
    print(f"taille : {round(os.path.getsize(image_path) / 1024, 2)} ko")

# Exemple d'utilisation
image_path = 'images/EFTA00003159-0.png'
display_image_metadata(image_path)

# Load JSON data from a file
data = json.load(open("data/image_metadata.json"))

# Convert JSON data to a pandas DataFrame
dataframe = pd.json_normalize(data)
for index, row in dataframe.iterrows():
    
    image_path = "images/" + row["id"]
    
    if os.path.exists(image_path):
        with Image.open(image_path) as img:
            paletteListe.append(img.palette)
            modeListe.append(img.mode)
            tailleListe.append(str(round(os.path.getsize(image_path) / 1024, 2)) + " ko")
    else:
        paletteListe.append("Image not found")
        modeListe.append("Image not found")
        tailleListe.append("Image not found")
        
dataframe["palette"] = paletteListe
dataframe["mode"] = modeListe
dataframe["size"] = tailleListe
dataframe



Métadonnées de l'image :
Format : PNG
Mode : P
dimension : (980, 1309)
Palette : <PIL.ImagePalette.ImagePalette object at 0x0000020C73A636D0>
Infos supplémentaires : {'dpi': (72.009, 72.009)}
taille : 499.83 ko


,id,original_filename,source,release_batch,source_url,redacted,width,height,person_ids,palette,mode,size
0,EFTA00004157-3.png,EFTA00004157-3.png,doj,NaN,https://www.justice.gov/epstein/files/DataSet%...,False,1240,1688,None,Image not found,Image not found,Image not found
1,EFTA00004157-5.png,EFTA00004157-5.png,doj,NaN,https://www.justice.gov/epstein/files/DataSet%...,False,1240,1688,None,Image not found,Image not found,Image not found
2,EFTA00004157-7.png,EFTA00004157-7.png,doj,NaN,https://www.justice.gov/epstein/files/DataSet%...,False,1240,1688,None,Image not found,Image not found,Image not found
3,EFTA00005381-1.png,EFTA00005381-1.png,doj,NaN,https://www.justice.gov/epstein/files/DataSet%...,False,1240,1688,None,Image not found,Image not found,Image not found
4,EFTA00005386-78.png,EFTA00005386-78.png,doj,NaN,https://www.justice.gov/epstein/files/DataSet%...,False,1240,1818,None,Image not found,Image not found,Image not found
...,...,...,...,...,...,...,...,...,...,...,...,...
95,EFTA00003236-2.png,EFTA00003236-2.png,doj,VOL00002,https://www.justice.gov/epstein/files/DataSet%...,False,2146,3020,None,<PIL.ImagePalette.ImagePalette object at 0x000...,P,14.95 ko
96,EFTA00003236-3.png,EFTA00003236-3.png,doj,VOL00002,https://www.justice.gov/epstein/files/DataSet%...,False,2260,3019,None,<PIL.ImagePalette.ImagePalette object at 0x000...,P,31.65 ko
97,EFTA00003236-4.png,EFTA00003236-4.png,doj,VOL00002,https://www.justice.gov/epstein/files/DataSet%...,False,2215,3023,None,<PIL.ImagePalette.ImagePalette object at 0x000...,P,137.08 ko
98,EFTA00003236-5.png,EFTA00003236-5.png,doj,VOL00002,https://www.justice.gov/epstein/files/DataSet%...,False,2169,3039,None,<PIL.ImagePalette.ImagePalette object at 0x000...,P,11.75 ko
